In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from torch.nn import Softplus #smooth relu
import torch.nn as nn
import torch.nn.functional as F
from utils import generate_polynomial_data
import numpy as np
import torch
import torch.optim as optim
from torch.nn import MSELoss
from tqdm import tqdm

from model import Net

In [11]:
suffix = "3"

n = 5000
d = 10000
generate_data = False

coeffs = np.random.rand(d, 1)
xvals = np.random.rand(n)
if generate_data:
    print("Generating Data...")
    X, Y = generate_polynomial_data(coeffs, xvals)
    '''with open("./datasets/X%s.npy" %suffix, "wb") as f:
        np.save(f, X)
    with open("./datasets/Y%s.npy" %suffix, "wb") as f:
        np.save(f, Y)
    with open("./datasets/coeffs%s.npy" %suffix, "wb") as f:
        np.save(f, coeffs)'''
else:
    with open("./datasets/X%s.npy" %suffix, "rb") as f:
        X = np.load(f)
    with open("./datasets/Y%s.npy" %suffix, "rb") as f:
        Y = np.load(f)
    with open("./datasets/coeffs%s.npy" %suffix, "rb") as f:
        coeffs = np.load(f)

In [12]:
net = Net(d, epochs = 20)

In [13]:
device = next(net.parameters()).device

In [14]:
X_cuda = torch.from_numpy(X).to(device)
Y_cuda = torch.from_numpy(Y).to(device)

In [ ]:
net.train_gd(X, Y, T=2000, lr=1e-03)

  1%|          | 22/2000 [00:00<00:09, 218.50it/s]

Epoch 1 iteration 2000 loss: 1187.38


 81%|████████▏ | 1625/2000 [00:07<00:01, 213.24it/s]

In [ ]:
suffix = "sgd_20_epochs"
torch.save(net.state_dict(), "./models/model%s.pt"%suffix)